In [11]:
# Load the full dataset of Chicago Crime
import pandas as pd
from pandas import DataFrame
import google
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Master-02851e90403f.json"

# Imports the Google Cloud client library
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client.from_service_account_json('Master-02851e90403f.json')

# The name for the new bucket
bucket_name = "bi-dataset-chicago"

from google.cloud import bigquery

client = bigquery.Client()
dataset_ref = client.dataset('chicago_crime', project='bigquery-public-data')
chicago_dataset = client.get_dataset(dataset_ref)

chicago_table = client.get_table(chicago_dataset.table('crime'))

# SQL query
# We reduce the total data to only 2019 non-domestic events, which is the target we want to study
query="""SELECT * FROM `bigquery-public-data.chicago_crime.crime` WHERE domestic=FALSE and year = 2019"""
# Set up the query
query_job = client.query(query)
#API request - run the query, and return a pandas DataFrame
chicago_crime = query_job.to_dataframe()

#Export to .csv
chicago_crime.to_csv('chicago_crime.csv', sep=',')

KeyboardInterrupt: 

In [3]:
# Clean the Dataset
# Load the .csv as Pandas Dataframe
import pandas as pd
chicago_crime = pd.read_csv("chicago_crime.csv")

# Change datetime to date
import datetime
from pandas import Series, DataFrame

s = chicago_crime['date'].str.split(' ').apply(Series, 1)
s.name = 'date'
del s[1]
del chicago_crime['date']
chicago_crime = chicago_crime.join(s)

# Name the date column
chicago_crime.columns = ['Unnamed: 0','unique_key','case_number','block','iucr','primary_type','description','location_description','arrest','domestic','beat','district','ward','community_area','fbi_code','x_coordinate','y_coordinate','year','updated_on','latitude','longitude','location','date']
del chicago_crime['Unnamed: 0']

# Export to .csv
chicago_crime.to_csv('chicago_crime.csv', sep=',')

In [4]:
chicago_crime

,unique_key,case_number,block,iucr,primary_type,description,location_description,arrest,domestic,beat,...,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location,date
0,11736749,JC323564,133XX S PRAIRIE AVE,502R,OTHER OFFENSE,VEHICLE TITLE/REG OFFENSE,STREET,True,False,533,...,54.0,26,1180389.0,1816938.0,2019,2019-07-03 16:13:23+00:00,41.652934,-87.615626,"(41.652933701, -87.615626241)",2019-06-26
1,11742960,JC331059,023XX S NORMAL AVE,0326,ROBBERY,AGGRAVATED VEHICULAR HIJACKING,STREET,False,False,914,...,34.0,03,1172986.0,1888549.0,2019,2019-07-19 16:09:50+00:00,41.849609,-87.640604,"(41.849609337, -87.640603567)",2019-07-02
2,11744041,JC332346,054XX N NEENAH AVE,1305,CRIMINAL DAMAGE,CRIMINAL DEFACEMENT,RESIDENCE,False,False,1613,...,10.0,14,1131923.0,1935662.0,2019,2019-07-19 16:09:50+00:00,41.979703,-87.790217,"(41.979703442, -87.7902173)",2019-07-02
3,11755670,JC346268,058XX N NORTHWEST HWY,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,NURSING HOME/RETIREMENT HOME,True,False,1611,...,10.0,02,1131920.0,1938442.0,2019,2019-08-12 16:08:26+00:00,41.987332,-87.790163,"(41.987332072, -87.790163497)",2019-07-12
4,11815598,JC417697,062XX S CICERO AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,AIRPORT EXTERIOR - SECURE AREA,False,False,813,...,64.0,24,1145449.0,1862707.0,2019,2019-09-09 16:18:42+00:00,41.779260,-87.742321,"(41.779260095, -87.742321037)",2019-09-02
5,11857750,JC468973,103XX S COTTAGE GROVE AVE,502P,OTHER OFFENSE,FALSE/STOLEN/ALTERED TRP,STREET,True,False,512,...,50.0,26,1182638.0,1836348.0,2019,2019-10-18 16:11:17+00:00,41.706146,-87.606799,"(41.706145802, -87.606798521)",2019-10-11
6,11865753,JC478469,093XX S STONY ISLAND AVE,0530,ASSAULT,AGGRAVATED: OTHER DANG WEAPON,STREET,False,False,413,...,48.0,04A,1188419.0,1843454.0,2019,2019-10-26 15:50:31+00:00,41.725510,-87.585403,"(41.72550966, -87.585403058)",2019-10-19
7,11900964,JC522148,106XX S BUFFALO AVE,0453,BATTERY,AGGRAVATED PO: OTHER DANG WEAP,STREET,True,False,432,...,52.0,04B,1199860.0,1835102.0,2019,2019-11-30 15:53:03+00:00,41.702311,-87.543776,"(41.702310644, -87.543775632)",2019-11-23
8,11908853,JC531806,070XX N ORIOLE AVE,1710,OFFENSE INVOLVING CHILDREN,ENDANGERING LIFE/HEALTH CHILD,RESIDENCE,False,False,1611,...,9.0,20,1124734.0,1946056.0,2019,2019-12-08 15:58:17+00:00,42.008348,-87.816425,"(42.008347647, -87.816425407)",2019-12-01
9,11908985,JC531942,040XX E 134TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,433,...,55.0,11,1204544.0,1816604.0,2019,2019-12-09 15:56:36+00:00,41.651431,-87.527258,"(41.651431329, -87.527258234)",2019-12-02


In [6]:
# Encode Type of Incident to Numbers
from sklearn import preprocessing as pr
 
chicago_crime = chicago_crime.drop(columns = ['location','iucr','beat','x_coordinate','y_coordinate','updated_on'], axis = 1)

In [7]:
# New DataFrame

chicago_crime.head()

,unique_key,case_number,block,primary_type,description,location_description,arrest,domestic,district,ward,community_area,fbi_code,year,latitude,longitude,date
0,11736749,JC323564,133XX S PRAIRIE AVE,OTHER OFFENSE,VEHICLE TITLE/REG OFFENSE,STREET,True,False,5,9.0,54.0,26,2019,41.652934,-87.615626,2019-06-26
1,11742960,JC331059,023XX S NORMAL AVE,ROBBERY,AGGRAVATED VEHICULAR HIJACKING,STREET,False,False,9,25.0,34.0,03,2019,41.849609,-87.640604,2019-07-02
2,11744041,JC332346,054XX N NEENAH AVE,CRIMINAL DAMAGE,CRIMINAL DEFACEMENT,RESIDENCE,False,False,16,45.0,10.0,14,2019,41.979703,-87.790217,2019-07-02
3,11755670,JC346268,058XX N NORTHWEST HWY,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,NURSING HOME/RETIREMENT HOME,True,False,16,41.0,10.0,02,2019,41.987332,-87.790163,2019-07-12
4,11815598,JC417697,062XX S CICERO AVE,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,AIRPORT EXTERIOR - SECURE AREA,False,False,8,23.0,64.0,24,2019,41.779260,-87.742321,2019-09-02


In [8]:
#Export to .csv
chicago_crime.to_csv('chicago_crime_clean.csv', sep=',')